In [27]:
import astropy
import math
import numpy as np
import scipy as sp
#import lmfit
import csv
from decimal import Decimal
import matplotlib.pyplot as plt
import matplotlib.artist as artists
from matplotlib.pyplot import savefig
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import ScalarFormatter 
from matplotlib.backends.backend_pdf import PdfPages
import scipy.optimize as optimization
#import cosmolopy.distance as cd
from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import Planck13, z_at_value
from astropy.table import Table
import astropy.units as u
import collections

In [3]:
cosmo = FlatLambdaCDM(H0=68, Om0=0.3)
print cosmo

FlatLambdaCDM(H0=68 km / (Mpc s), Om0=0.3, Tcmb0=0 K, Neff=3.04, m_nu=None, Ob0=None)


In [10]:
# Calculate dispersion measure from redshift - from coherent theory paper eqn 2
def dm(z):
    return 1200. * z

In [53]:
# Redshift numpy array
redshift = np.linspace(0.,1.,100,endpoint=True)
# Array for results
ddata_results = np.ndarray(100) 
ddata_sensitivity = np.ndarray(100) 

In [21]:
# Equations from coherent paper equation 1 (in seconds)
# Frequency dependent delay using DM (d in pc cm^-3) and frequency (f in GHz)
def dta(z, f):
    d = 1200. * z
    return d / (241. * f**2)

In [22]:
# Equations from Trott et al. (2013), ApJL, 776, L16 (in ms) - eqn 1
# Calculate dispersion delay using DM (d in pc cm^-3) and frequency (f in Hz or GHz??)
def dtt(z, f):
    d = 1200. * z
    return 4.15 * (d / f**2)

In [71]:
# Equations from Trott et al. (2013), ApJL, 776, L16 (in micro s) - eqn 8
# Assumes zero scattering
# Dispersion delay across a single channel (in us) assuing 
# a dispersion measure of DM (d in pc cm^-3) 
# an observing frequency (f in GHz) 
# and a spectral resolution (v in MHz)
def dtdm(z,f,v):
    d = 1200. * z
    return 8.3 * (d / f**3) * v

In [72]:
# Equations from Trott et al. (2013), ApJL, 776, L16 (in micro s) - eqn 7
# Assumes zero scattering
# Signal observed by the system has the characteristic timescale (in micro s) assuming 
# a pulse intrinsic timescale of (w in micro s)
# a dispersion measure of DM (d in pc cm^-3) 
# an observing frequency (f in GHz) 
# and a spectral resolution (v in MHz)
def wobs(w,z,f,v):
    d = 1200. * z
    tdm = 8.3 * (d / f**3) * v
    return np.sqrt(w**2 + tdm**2)

In [73]:
# Create arrays of dispersion delay and observed timescale
# Assume the pulse intrinsic timescale is 1000 micro s and the temporal sampling timescale (0.5s = 5e5 ms)
wint = 1000.
tsamp = 5e5
# Sensitivity of MWA (3 sigma in Jy)
smwa = 3.
# Array of observed dispersion delays for dedispersed imaging
#ddatat = dtdm(redshift,0.185,1.28)
ddataw = wobs(wint,redshift,0.185,1.28)
#bdatat = dtdm(redshift,0.185,30.72)
bdataw = wobs(wint,redshift,0.185,30.72)

In [74]:
# tests
#ddatat
#ddataw
#bdatat
#bdataw

In [75]:
# Create boolean
# Filter array for elements in data1
# Comparisons between the sampling timescale (tsamp) and the intrinsic pulse timescale (wint)
fltr_a = (ddataw < tsamp)
fltr_b = (wint > tsamp)
fltr_c = ((ddataw > tsamp) & (wint < tsamp))
print(fltr_a)
print(fltr_b) # This is checking the overall inequality - this will always be false in imaging mode!
print(fltr_c)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
False
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True 

In [76]:
ddata_results[fltr_c]

array([0.00175299, 0.00171895, 0.00168682, 0.00165642, 0.00162761,
       0.00160026, 0.00157423, 0.00154944, 0.00152579, 0.00150318,
       0.00148155, 0.00146083, 0.00144095, 0.00142187, 0.00140352,
       0.00138586, 0.00136886, 0.00135246, 0.00133665, 0.00132137,
       0.0013066 , 0.00129232, 0.0012785 , 0.00126511, 0.00125214,
       0.00123955, 0.00122734, 0.00121548, 0.00120396, 0.00119276,
       0.00118187, 0.00117127, 0.00116095, 0.0011509 , 0.0011411 ,
       0.00113155, 0.00112224, 0.00111315, 0.00110428, 0.00109562,
       0.00108716, 0.00107889, 0.00107081, 0.00106291, 0.00105518,
       0.00104761, 0.00104021, 0.00103296, 0.00102586, 0.00101891,
       0.00101209, 0.00100541, 0.00099886, 0.00099244, 0.00098614,
       0.00097995, 0.00097389, 0.00096793, 0.00096208, 0.00095634,
       0.00095069, 0.00094515, 0.0009397 , 0.00093435, 0.00092909,
       0.00092391, 0.00091882, 0.00091381, 0.00090889, 0.00090404,
       0.00089927, 0.00089457, 0.00088995, 0.0008854 , 0.00088

In [79]:
# For elements of ddataw, which the a filter is true, multiply telescope sensitivity by wint/tsamp
ddata_results[fltr_a] = (wint/tsamp) 
# For elements of ddataw, which the c filter is true, multiply telescope sensitivity by wint/sqrt(wobs * tsamp)
# For elements of data1 which filter is NOT true (hence the "~" character), multiply by -10
##data1_results[~fltr] = data1[~fltr] * -10
ddata_results[~fltr_a] = (wint / np.sqrt(ddataw[~fltr_a] * tsamp) )

ddata_sensitivity = 1. / ddata_results

print(redshift)
print(ddata_results)
print(ddata_sensitivity)


#data1_results[~fltr] = data1[~fltr] * -10

#print(ddataw_results_a_true)

[0.         0.01010101 0.02020202 0.03030303 0.04040404 0.05050505
 0.06060606 0.07070707 0.08080808 0.09090909 0.1010101  0.11111111
 0.12121212 0.13131313 0.14141414 0.15151515 0.16161616 0.17171717
 0.18181818 0.19191919 0.2020202  0.21212121 0.22222222 0.23232323
 0.24242424 0.25252525 0.26262626 0.27272727 0.28282828 0.29292929
 0.3030303  0.31313131 0.32323232 0.33333333 0.34343434 0.35353535
 0.36363636 0.37373737 0.38383838 0.39393939 0.4040404  0.41414141
 0.42424242 0.43434343 0.44444444 0.45454545 0.46464646 0.47474747
 0.48484848 0.49494949 0.50505051 0.51515152 0.52525253 0.53535354
 0.54545455 0.55555556 0.56565657 0.57575758 0.58585859 0.5959596
 0.60606061 0.61616162 0.62626263 0.63636364 0.64646465 0.65656566
 0.66666667 0.67676768 0.68686869 0.6969697  0.70707071 0.71717172
 0.72727273 0.73737374 0.74747475 0.75757576 0.76767677 0.77777778
 0.78787879 0.7979798  0.80808081 0.81818182 0.82828283 0.83838384
 0.84848485 0.85858586 0.86868687 0.87878788 0.88888889 0.89898

In [26]:
# Tests - calculate the dispersion delay timescale and the system characteristic timescale
# Assume w = 1000 micro s, f = 0.185 GHz, v = 1.28 MHz (image de-dispersion)
# Assume w = 1000 micro s, f = 0.185 GHz, v = 30.72 MHz (whole MWA band)
# DM = 840.0 corresponding to z = 0.7 
print dtdm(0.7,0.185,1.28)
print wobs(1000.,0.7,0.185,1.28)
print dtdm(0.7,0.185,30.27)
print wobs(1000.,0.7,0.185,30.72)

1804106.33921
1804106.6163554231
1008943937.58
1039165251.3853942


In [20]:
dmgw = dm(zgw)
ddgw = dd(dmgw,0.15)

In [21]:
print dmgw
print ddgw

31.9064207794
5.88407944295


In [22]:
dmgwbns = dm(zgwbns)
ddgwbns = dd(dmgwbns,0.15)

In [23]:
print dmgwbns
print ddgwbns

10.776572388
1.98738079999


In [24]:
print cjdd(120,150)
print cjddh(120,150)

print cjdd(40,150)
print cjddh(40,150)

14.976
26.0693333333
11.0933333333
22.1866666667


In [31]:
print cjddb(4384.6844,185)

119.284506973


In [38]:
print cjddb(40,185)

7.2972972973


In [34]:
print cjddb(4384.6844,150)

181.44498894


In [39]:
print cjddb(40,150)

11.1


In [36]:
print cjddb(4384.6844,132.5)

232.53896767


In [40]:
print cjddb(40,132.5)

14.2257030972


In [33]:
print cjdd(4384.6844,185)

119.212864626


In [41]:
print cjdd(40,185)

7.29291453616


In [35]:
print cjdd(4384.6844,150)

181.336012971


In [42]:
print cjdd(40,150)

11.0933333333


In [37]:
print cjdd(4384.6844,132.5)

232.399304626


In [43]:
print cjdd(40,132.5)

14.2171591314


In [30]:
print cjddb(40,150)

11.1
